In [88]:
MG = rg.read('graphs/sp_final.gexf')

In [4]:
def getpath(nested_dict, value, prepath=()):
    for k, v in nested_dict.items():
        path = prepath + (k,)
        if v == value: # found value
            return path
        elif hasattr(v, 'items'): # v is a dict
            p = getpath(v, value, path) # recursive call
            if p is not None:
                return p

In [109]:
node_id = '706304'
walker = Walker(MG, node_id, travel_time=60, current_time=7)

walker.random_isochrone()


440016923
70016924
440015046
70001989
Todos locais passados:  ['440016923']
440016923
920015297
70016834
706304
70001989
9206535
706309
706308
440015046
9206520
18858
Todos locais passados:  ['440016923', '706304']
4406614
18859
70003167
Todos locais passados:  ['440016923', '706304', '70016916']
260016919
440016922
70016918
70003263
260016919
440016922
70016918
70003263
Todos locais passados:  ['440016923', '706304', '70016916', '70016918']
260016881
260016855
2600672
18850
260016780
260015937
440015041
4406625


KeyError: 'viagens'

In [116]:
node_id = '4406625'

walker.MG.node[node_id]

walker.MG.edge[node_id]

walker.MG.

#walker.MG.node[node_id]['viagens'].keys()

{'260016780': {0: {'attr': {'tempo': 279, 'trip_id': 'CT01-1-0'},
   'id': '69411'}}}

In [57]:
import math
import networkx as nx
import scripts.read_graph as rg
import folium
import random
import time
import numpy


def get_distance(lon1, lat1, lon2, lat2):
    
    R = 6373.0

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    
    return distance*1000.0 #em M, por isso multiplica por 1000.0

#0.002 é uma caixa de 200-300m +-
    

In [106]:
class Walker():
    
    def __init__(self, graph, stop_id, travel_time, current_time):
        self.MG = graph 
        self.stop_id = stop_id 
        self.total_time_travel = travel_time * 60 # minutos para segundos 
        self.traveled_time = 0 
        self.visited_set = [] 
        self.tentative_value = {} 
        self.current_time = current_time * 60 * 60 # hora para segundo 
        self.walking_speed = 1 #em m/s
        
        self.calculate_location = lambda x: (MG.node[x]['location']['lat'], MG.node[x]['location']['long'])# tuple
        self.current_location = self.calculate_location(stop_id)
    
    def restart(self, current_time, stop_id):
        self.traveled_time = 0
        self.current_time = current_time
        self.stop_id = stop_id
    
    def look_around(self,maximum_dist):
        '''
        Look at current stop coordinates and find close stops
        Receives a maximum distance (in degrees) and 
        returns a dictionary with close stops and distance (in meters) to each of them
        '''
        close_stops = {}
        
        long_max = self.MG.node[self.stop_id]['location']['long'] + maximum_dist
        long_min = self.MG.node[self.stop_id]['location']['long'] - maximum_dist
        
        lat_max = self.MG.node[self.stop_id]['location']['lat'] + maximum_dist
        lat_min = self.MG.node[self.stop_id]['location']['lat'] - maximum_dist
        
        for stops in self.MG.node.keys():
            
            if (self.MG.node[stops]['location']['long']<=long_max and
                self.MG.node[stops]['location']['long']>=long_min and
                self.MG.node[stops]['location']['lat']<=lat_max and
                self.MG.node[stops]['location']['lat']>=lat_min):
                
                
                close_stops[stops] = get_distance(
                    self.MG.node[self.stop_id]['location']['long'],
                    self.MG.node[self.stop_id]['location']['lat'],
                    self.MG.node[stops]['location']['long'],
                    self.MG.node[stops]['location']['lat'])

        del close_stops[self.stop_id]
        return(close_stops)
    
    def build_map(self):
        
        stops = [self.MG.node[i] for i in self.visited_set]
        COORD = (stops[0]['location']['lat'],stops[0]['location']['long']) # sp coordinates
        
        mapbox_access_token = 'pk.eyJ1IjoiY2hlbHNlYXBsb3RseSIsImEiOiJjaXFqeXVzdDkwMHFrZnRtOGtlMGtwcGs4In0.SLidkdBMEap9POJGIe1eGw' 
        
        data = Data([
            Scattermapbox(
                lat=[s['location']['lat'] for s in stops],
                lon=[s['location']['long'] for s in stops],
                mode='markers',
                marker=Marker(
                    size=9
                )
            )
        ])
        layout = Layout(
            autosize=True,
            hovermode='closest',
            mapbox=dict(
                accesstoken=mapbox_access_token,
                bearing=0,
                center=dict(
                    lat=COORD[0],
                    lon=COORD[1]
                ),
                pitch=0,
                zoom=10
            ),
        )

        fig = dict(data=data, layout=layout)
        return fig
    
        """
        mapa = folium.Map(location=COORD, zoom_start=12)
        
        for i, stop in enumerate(stops[1:]):
            
            if i == 0:
                color = 'black'
                fill_color = 'red'
                radius=5
            elif i < len(self.visited_set):
                fill_color = 'black'
                color = 'black'
                radius=1
                

            folium.RegularPolygonMarker(
                location = [stop['location']['lat'],stop['location']['long']],
                color=color,
                fill_color=fill_color,
                number_of_sides=100,
                radius=radius,
                ).add_to(mapa)

        return(mapa)
        """
    def isochrone(self):
        i=0
        while 1:
            #print('\n{}: '.format(i))
            self.find_neighbours()

            minimum, min_key = -1, 0
            for key, values in self.tentative_value.items():
                if key not in self.visited_set:
                    if minimum == -1:
                        minimum = values['travel_time']
                        min_key = key
                    elif values['travel_time'] < minimum:
                        minimum = values['travel_time']
                        min_key = key
            
            
            self.stop_id = min_key
            self.traveled_time = self.traveled_time + minimum
            self.current_time = self.current_time + minimum
            
            #print('\n--------\nBusao:', self.tentative_value[min_key]['linha'])
            #print('Tentative_Values', self.tentative_value)
            #print('Visited set', self.visited_set)
            #print('Traveled Time: ',self.traveled_time)
            if self.traveled_time > self.total_time_travel:
            #    print('FINAL:', self.stop_id)
                break
            i=i+1
    
        
            
    def find_neighbours(self):
        """Given a stop, find neighbours and calculate travel time. 
        Two types of neighbours: connectionable and walkable
        """
        #print('StopId: ', self.stop_id)
        #pprint(self.current_time)
        #get connection neighbours
        linha_atual = 0
        if len(self.visited_set) > 0:
            linha_atual = self.tentative_value[self.stop_id]['linha']
            # print('Linha Atual:', linha_atual)
            
        for conn_neigh_key, conn_neigh_value in self.MG.node[self.stop_id]['viagens'].items():
            
            #print('TripId', conn_neigh_key)
            # get next travel time
            # print("------\nLine Name: ", conn_neigh_key, "\n")
            #print(conn_neigh_value)
            travel_time =  (min(conn_neigh_value,
                              key=lambda x: abs(x-self.current_time) if x-self.current_time<=0 else False) -
                            self.current_time)
            """
            if conn_neigh_key == linha_atual:
                pass
                print("----Current Line:", travel_time)
            else:
                pass
                print("Other Line:", travel_time)
            """           
            # have to be greater than current_time
            if travel_time < 0:
                continue
            
            # get time to next stop  
            output = getpath(self.MG.edge[self.stop_id], conn_neigh_key)
            if output == None:
                continue
            stop_id_next, trip_key, *rest = output
            
            if stop_id_next in self.visited_set:
                continue
            trip_time = self.MG.edge[self.stop_id][stop_id_next][trip_key]['attr']['tempo']
            travel_time = travel_time + trip_time
            """
            print("Trip Time: ", trip_time)
            print("Total Time: ", travel_time)
            print('Stop Next', stop_id_next, "\n")
            """
            # check travel time to next stop and replace if smaller
            if stop_id_next in self.tentative_value:
                if travel_time < self.tentative_value[stop_id_next]['travel_time']:
                    self.tentative_value[stop_id_next] = {'travel_time': travel_time,
                                                          'trip_key': trip_key,
                                                          'previous_stop': self.stop_id,
                                                        'linha': conn_neigh_key}
            else:
                self.tentative_value[stop_id_next] = {'travel_time': travel_time,
                                                      'trip_key': trip_key,
                                                      'previous_stop': self.stop_id,
                                                     'linha': conn_neigh_key}
   
            #print(self.tentative_value)
    
        self.visited_set.append(self.stop_id)
        
        # get walkable neighbours
        
    def random_isochrone(self):
        tt = 0
        ot = 0
        while 1:
            
            ###Olha todas as linhas e stops em uma caixa ao redor
            ###Depois disso, olha as linhas na stop atual
            ###A seguir, junta todas as stops e linhas em um dicionario só
            
            close_stops = self.look_around(0.002)
            linhas_andando = []
            stops_linhas_andando = {}
            linhas_na_stop = {}
            
            for key in close_stops.keys():
                stops_linhas_andando[key] = []
                
            for key in close_stops.keys():
                print (key)
                stops_linhas_andando[key] = stops_linhas_andando[key] + (list(self.MG.node[key]['viagens'].keys()))
            
            linhas_na_stop[self.stop_id] = list(self.MG.node[self.stop_id]['viagens'].keys())
                
            linhas_totais = {**linhas_na_stop, **stops_linhas_andando}

            #print("Linhas na stop: ", linhas_na_stop)
            #print("Linhas andando: ", stops_linhas_andando)
            #print("Linhas totais: ", linhas_totais)
            
            ### A partir do total de linhas, da peso para cada stop proxima conforme o numero de linhas em cada
            
            count_linhas = []
            for key in linhas_totais.keys():
                count_linhas.append(len(linhas_totais[key]))
            count_total_linhas = sum(count_linhas)
            
            pesos = []
            
            for key in linhas_totais.keys():
                pesos.append(len(linhas_totais[key])/count_total_linhas)
                
            ###Escolhe uma stop conforme o peso e em seguida uma linha aleatoria
            
            stop_aleatoria = numpy.random.choice(list(linhas_totais.keys()), p=pesos)
            linhas_aleatorio = numpy.random.choice(linhas_totais[stop_aleatoria])
            
            ##Se andou ate outra stop, adiciona o tempo gasto andando e atualiza a stop
            if stop_aleatoria != self.stop_id:
                
                distancia = close_stops[stop_aleatoria]
                tempo_andando = distancia/self.walking_speed
                
                tempo_andando = close_stops[stop_aleatoria]/self.walking_speed
                self.stop_id = stop_aleatoria
                self.traveled_time = self.traveled_time + tempo_andando
                self.current_time = self.current_time + tempo_andando
                
            ### Pega todos os horarios da linha escolhida e a seguir quando vai passar o proximo busao/metro
        
            valores_linhas_aleatorio = (self.MG.node[self.stop_id]['viagens'][linhas_aleatorio])

            travel_time =  (min(valores_linhas_aleatorio,
                                  key=lambda x: abs(x-self.current_time) if x-self.current_time<=0 else False) -
                                self.current_time)

            # have to be greater than current_time
            if travel_time < 0:
                tt = tt + 1
                if tt > 60:
                    break
                #print('travel_time')
                continue

            # get time to next stop  
            output = getpath(self.MG.edge[self.stop_id], linhas_aleatorio)
            if output == None:
                #print('Stop Id: ', self.stop_id)
                #print('Linha: ', linhas_aleatorio)
                ot = ot + 1
                if ot > 60:
                    break
                #print('output')
                continue

            stop_id_next, trip_key, *rest = output

            trip_time = self.MG.edge[self.stop_id][stop_id_next][trip_key]['attr']['tempo']
            travel_time = travel_time + trip_time

            #print("Trip Time: ", trip_time)
            #print("Total Time: ", travel_time)
            #print('Stop Next', stop_id_next, "\n")

            self.visited_set.append(self.stop_id)

            self.stop_id = stop_id_next
            self.traveled_time = self.traveled_time + trip_time
            self.current_time = self.current_time + trip_time

            print("Todos locais passados: ", self.visited_set)
            #print("Atual: ",self.stop_id)
            #print("Tempo Total: ",self.traveled_time)

            if self.traveled_time > self.total_time_travel:
                #print('FINAL:', self.stop_id)
                break 